<a href="https://colab.research.google.com/github/Palanisamy-Selvaraj/Palanisamy-Selvaraj/blob/main/ETC_Code1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%% Data-Driven Control using DMDc (Dynamic Mode Decomposition with Control)
clear; clc; close all;

% 1. Setup Parameters
dt = 0.01;          % Time step
T = 5;              % Total time
t = 0:dt:T;
n_states = 2;       % Number of states (e.g., position, velocity)
n_inputs = 1;       % Number of control inputs

% 2. Generate Training Data (Random Inputs)
% We use random inputs to "excite" the system and learn its dynamics.
u_train = randn(n_inputs, length(t)-1);
X = zeros(n_states, length(t));
X(:,1) = [1; 0]; % Initial condition

% Simulated Nonlinear System (The "True" System)
% x_dot = [x2; -0.5*x2 - sin(x1) + u]
for i = 1:length(t)-1
    x_curr = X(:,i);
    u_curr = u_train(:,i);

    % Simple Euler integration for the nonlinear dynamics
    x_next = x_curr + dt * [x_curr(2); -0.5*x_curr(2) - sin(x_curr(1)) + u_curr];
    X(:,i+1) = x_next;
end

% Prepare DMDc Matrices
X1 = X(:, 1:end-1); % Current states
X2 = X(:, 2:end);   % Next states (shifted by 1)
Upsilon = u_train;  % Inputs used

% 3. Perform DMDc (System Identification)
% Solve for [A, B] such that X2 = A*X1 + B*Upsilon
% Omega = [X1; Upsilon]
Omega = [X1; Upsilon];
G = X2 * pinv(Omega); % Least-squares fit

A_hat = G(:, 1:n_states);
B_hat = G(:, n_states+1:end);

fprintf('DMDc Identification Complete.\n');

% 4. Control Design (LQR)
% Penalize state deviation (Q) and control effort (R)
Q = eye(n_states) * 100;
R = 1;
[K, ~, ~] = dlqr(A_hat, B_hat, Q, R); % Discrete LQR gain

% 5. Test the Data-Driven Controller
x_test = zeros(n_states, length(t));
x_test(:,1) = [2; 0.5]; % Different initial condition
u_applied = zeros(n_inputs, length(t)-1);

for i = 1:length(t)-1
    % Compute control law: u = -Kx
    u_applied(:,i) = -K * x_test(:,i);

    % Evolve the "True" Nonlinear System with the data-driven controller
    x_curr = x_test(:,i);
    u_curr = u_applied(:,i);
    x_next = x_curr + dt * [x_curr(2); -0.5*x_curr(2) - sin(x_curr(1)) + u_curr];
    x_test(:,i+1) = x_next;
end

% 6. Plot Results
figure;
subplot(2,1,1);
plot(t, x_test(1,:), 'LineWidth', 2); hold on;
plot(t, x_test(2,:), 'LineWidth', 2);
title('State Trajectories (Data-Driven LQR Control)');
legend('Position (x1)', 'Velocity (x2)');
grid on;

subplot(2,1,2);
plot(t(1:end-1), u_applied, 'r', 'LineWidth', 1.5);
title('Control Input (u)');
xlabel('Time (s)');
grid on;

UsageError: Cell magic `%%` not found.
